### Create ragas TestsetGenerator


#### import modules and setup env

In [1]:
import os
import random

from typing import List
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.docstore.document import Document
import pandas as pd

os.chdir("../../")
from src.rag_pipeline.load_docs import load_docs_from_csv
from src.env_loader import load_api_keys

load_api_keys()


#### Define llms

In [2]:
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

#### Define generator and distributions

In [3]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 0.5,
    multi_context: 0.25,
    reasoning: 0.25
}

In [4]:
documents = load_docs_from_csv(as_document=True)

#### Get a random sample of the documents

In [9]:

def select_random_documents(documents: List[Document], num: int) -> List[Document]:
    """
    Select a specified number of randomly chosen documents from a list.

    Args:
        documents (List[Document]): A list of Document objects (expected to be 1000 in length).
        num (int): The number of documents to select randomly.

    Returns:
        List[Document]: A list of randomly selected Document objects.

    Raises:
        ValueError: If num is greater than the length of the documents list or if documents list is empty.
    """
    if not documents:
        raise ValueError("The documents list is empty.")
    
    if num > len(documents):
        raise ValueError("The number to select cannot be greater than the number of documents available.")
    
    if num < 0:
        raise ValueError("The number to select cannot be negative.")

    selected_documents = random.sample(documents, num)
    return selected_documents

In [10]:
selected_docs = select_random_documents(documents, 20)


In [ ]:
selected_docs

In [7]:
from typing import List
from langchain.docstore.document import Document

def get_random_document_sample(documents: List[Document], sample_size: int = 3) -> List[Document]:
    """Shuffles a list of Document objects and returns a random sample.

    Args:
        documents (List[Document]): The list of Document objects to sample from.
        sample_size (int, optional): The size of the sample to return. Defaults to 3.

    Returns:
        List[Document]: A list of Document objects, shuffled and sampled.
    """
    
    # Create a copy to avoid modifying the original list
    shuffled_docs = documents.copy()  

    # Shuffle the copy
    random.shuffle(shuffled_docs)  

    # Return a sample
    return shuffled_docs[:sample_size]

In [8]:
## Get a sample
sample_docs = get_random_document_sample(documents, sample_size=20)

In [9]:
len(sample_docs)
sample_docs

10

#### Generate the testset

In [11]:
# generate testset
testset = generator.generate_with_langchain_docs(selected_docs, test_size=20, distributions=distributions, raise_exceptions=False)

embedding nodes:   0%|          | 0/52 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/20 [00:00<?, ?it/s]

max retries exceeded for ReasoningEvolution(generator_llm=LangchainLLMWrapper(run_config=RunConfig(timeout=60, max_retries=15, max_wait=90, max_workers=16, thread_timeout=80.0, exception_types=<class 'openai.RateLimitError'>, log_tenacity=False)), docstore=InMemoryDocumentStore(splitter=<langchain_text_splitters.base.TokenTextSplitter object at 0x70acbf886620>, nodes=[Node(metadata={'source': 'cnn_dailymail', 'id': 'b08bab7ef9179d339ee2c4db0f59cd611228638b'}, page_content='(CNN)After decades in private hands, a spaceship from the classic film "2001: A Space Odyssey" will go on display for the first time to the public. The Academy of Motion Picture Arts and Sciences, the group that presents the Oscars, purchased the spacecraft at auction Saturday for $344,000. The academy will display the ship in its museum of movie history, scheduled to open in 2017 in Los Angeles. "We\'re excited to add such an iconic piece of movie history to our collection," the academy told CNN in a statement. "We 

In [12]:
testset

TestDataset(test_data=[DataRow(question='What is the significance of cherry trees in Washington, D.C.?', contexts=['Washington (CNN)There should be plenty of pent-up anticipation among cherry blossom fans in Washington this spring. Peak blooms are expected at the tail end of the three-week-long National Cherry Blossom Festival, which kicks off Friday and runs through April 12. The fluttering pink and white blossoms are expected to hit their peak between April 11 and April 14. Here are five things to know about the eagerly awaited D.C. blossoms: . The city\'s first cherry trees were destroyed . In 1910, 2,000 cherry trees arrived in Washington, thanks to a gift from the city of Tokyo and the advocacy of Eliza Ruhamah Scidmore, one of the first female board members of the National Geographic Society. These trees, however, were riddled with disease and were destroyed. Two years later, first lady Helen Herron Taft planted one of the first of more than 3,000 replacement trees sent from Japa

In [13]:
## Convert the test set to pandas dataframe
df = testset.to_pandas()
df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the significance of cherry trees in Wa...,[Washington (CNN)There should be plenty of pen...,Cherry trees hold great significance in Washin...,simple,"[{'source': 'cnn_dailymail', 'id': '1e84a98d5b...",True
1,What is one of the events that Hillary Clinton...,"[(CNN)After a handful events in two months, Hi...",Hillary Clinton will be participating in the A...,simple,"[{'source': 'cnn_dailymail', 'id': 'f04ad85fb2...",True
2,What role did the United States-led coalition ...,[(CNN)In his first interview since Islamic Sta...,The United States-led coalition played a role ...,simple,"[{'source': 'cnn_dailymail', 'id': 'e7f48a4a58...",True
3,What can viewers expect from the season finale...,[(CNN)They thought they had found a paradise. ...,"The season finale of ""The Walking Dead"" is exp...",simple,"[{'source': 'cnn_dailymail', 'id': 'a4c99a65c0...",True
4,What is the significance of the Tunisian jihad...,[(CNN)ISIS apparently claimed responsibility T...,The significance of the Tunisian jihadist in r...,simple,"[{'source': 'cnn_dailymail', 'id': '37b24ca5e6...",True
5,How did the undercover FBI informant play a ro...,[(CNN)An Army National Guard member and his co...,The undercover FBI informant played a role in ...,simple,"[{'source': 'cnn_dailymail', 'id': '3b89d00750...",True
6,How does the new Red brand aim to leverage Rad...,"[ a new Red brand that will, according to Ande...",The new Red brand aims to leverage Radisson's ...,simple,"[{'source': 'cnn_dailymail', 'id': '8f83a15fb3...",True
7,When are the peak blooms expected during the N...,[Washington (CNN)There should be plenty of pen...,The peak blooms are expected between April 11 ...,simple,"[{'source': 'cnn_dailymail', 'id': '1e84a98d5b...",True
8,What is Zhanna Nemtsova's opinion on the Russi...,"[Moscow (CNN)Russia's president shares ""politi...",Zhanna Nemtsova said she has no faith in the R...,simple,"[{'source': 'cnn_dailymail', 'id': '2f9a8fb69e...",True
9,Who was one of Putin's harshest critics?,"[Moscow (CNN)Russia's president shares ""politi...",Boris Nemtsov,simple,"[{'source': 'cnn_dailymail', 'id': '2f9a8fb69e...",True


#### Save the test set to a CSV

In [14]:
# Save to a CSV file
df.to_csv("data/evaluation_sets/evaluation_set_20d20.csv")  # index=False to avoid saving row numbers